【前提】
* 毎週金曜日に市場が閉じてからポートフォリオを構築する(初日（月曜日）の始値で購入、最終日（金曜日）の終値で売却)
* （要件1）原資100万円のうち、50万円以上株の購入に充てられていること
* （要件2）購入銘柄数が5銘柄以上であること

【戦略】
1. 直近の1週間のニュースを取得し、BERTモデルにより特徴量に変換
2. 1.で算出した各銘柄の特徴量をK-meansによるクラスタリングを行う。
3. LSTMモデルによりユニバース(投資対象銘柄群)と各業界のセンチメントスコアを算出  
   ※各スコアは学習期間のパラメータ(平均, 標準偏差)に基づき基準化したものにする
4. 3.のユニバースに対するスコアから現金比率を決定する。  
5. 2.のクラスタリングの結果、クラスター7に属する銘柄に投資する。  
   投資比率は3.の各業界のセンチメントスコアからセクター(17区分)から決定する。  
   ※セクター2, 11, 12, 15以外は投資対象から除外する
5. 各セクターの投資比率に基づき、投資対象銘柄を選定  
   * 直近1週間のニュースとして取り上げられた銘柄を対象とする。
   * 直近のニュースであるから優先的に選ぶ(諦めた)  
   (各銘柄の予測スコア(前回のファンダメンタル分析におけるコンペのモデル等)に基づき銘柄を選定したほうがよいかも?)  
   (もしくは、ラグが各業界のインデックスと相関があるような銘柄だとよいのかも?）

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import zscore

import sys, os, pickle, io
sys.path.append('../src/')
from SentimentGenerator import SentimentGenerator

In [2]:
class ScoringService(object):
    # テスト期間開始日
    TEST_START = "2021-02-01"
    # データをこの変数に読み込む
    dfs = None
    # モデルをこの変数に読み込む
    models = None
    # 対象の銘柄コードをこの変数に読み込む
    codes = None
    # センチメントの分布をこの変数に読み込む
    df_sentiment_dist = None

    @classmethod
    def get_dataset(cls, inputs, load_data):
        """
        Args:
            inputs (list[str]): path to dataset files
        Returns:
            dict[pd.DataFrame]: loaded data
        """
        if cls.dfs is None:
            cls.dfs = {}
        for k, v in inputs.items():
            # 必要なデータのみ読み込みます
            if k not in load_data:
                continue
            cls.dfs[k] = pd.read_csv(v)
            # DataFrameのindexを設定します。
            if k == "stock_price":
                cls.dfs[k].loc[:, "datetime"] = pd.to_datetime(
                    cls.dfs[k].loc[:, "EndOfDayQuote Date"]
                )
                cls.dfs[k].set_index("datetime", inplace=True)
            elif k in ["stock_fin", "stock_fin_price", "stock_labels"]:
                cls.dfs[k].loc[:, "datetime"] = pd.to_datetime(
                    cls.dfs[k].loc[:, "base_date"]
                )
                cls.dfs[k].set_index("datetime", inplace=True)
        return cls.dfs
    
    @classmethod
    def get_codes(cls, dfs):
        """
        Args:
            dfs (dict[pd.DataFrame]): loaded data
        Returns:
            array: list of stock codes
        """
        stock_list = dfs["stock_list"].copy()
        # 予測対象の銘柄コードを取得
        cls.codes = stock_list[stock_list["universe_comp2"] == True][
            "Local Code"
        ].values
        return cls.codes
    
    @classmethod
    def get_model(cls, model_path="../model", labels=None):
        """Get model method

        Args:
            model_path (str): Path to the trained model directory.
            labels (arrayt): list of prediction target labels

        Returns:
            bool: The return value. True for success, False otherwise.

        """
        if cls.models is None:
            cls.models = {}
        m = os.path.join(model_path, "kmeans_model.pickle")
        with open(m, "rb") as f:
            # pickle形式で保存されているモデルを読み込み
            cls.models['kmeans'] = pickle.load(f)
        
        # SentimentGeneratorクラスの初期設定を実施
        SentimentGenerator.initialize(model_path)
        
        # 事前に計算済みのセンチメントを分布として使用するために読み込みます
        cls.df_sentiment_dist = cls.load_sentiments(
            f"{model_path}/headline_features/LSTM_sentiment.pkl"
        )

        return True
    
    @classmethod
    def transform_yearweek_to_monday(cls, year, week):
        """
        ニュースから抽出した特徴量データのindexは (year, week) なので、
        (year, week) => YYYY-MM-DD 形式(月曜日) に変換します。
        """
        for s in pd.date_range(f"{year}-01-01", f"{year}-12-31", freq="D"):
            if s.week == week:
                # to return Monday of the first week of the year
                # e.g. "2020-01-01" => "2019-12-30"
                return s - pd.Timedelta(f"{s.dayofweek}D")
    
    @classmethod
    def load_sentiments(cls, path=None):
        #DIST_END_DT = "2020-09-25"

        print(f"[+] load prepared sentiment: {path}")

        # 事前に出力したセンチメントの分布を読み込み
        df_sentiments = pd.read_pickle(path)

        # indexを日付型に変換します変換します。
        df_sentiments.loc[:, "index"] = df_sentiments.index.map(
            lambda x: cls.transform_yearweek_to_monday(x[0], x[1])
        )
        # indexを設定します
        df_sentiments.set_index("index", inplace=True)
        
        # 金曜日日付に変更します
        df_sentiments.index = df_sentiments.index + pd.Timedelta("4D")

        return df_sentiments
    
    @classmethod
    def get_sentiment(cls, inputs, start_dt="2020-12-31"):
        # ニュース見出しデータへのパスを指定
        article_path = inputs["nikkei_article"]
        target_feature_types = ["headline"]
        df_sentiments = SentimentGenerator.generate_lstm_features(
            article_path,
            start_dt=start_dt,
            target_feature_types=target_feature_types,
        )["headline_features"]

        df_sentiments.loc[:, "index"] = df_sentiments.index.map(
            lambda x: cls.transform_yearweek_to_monday(x[0], x[1])
        )
        df_sentiments.set_index("index", inplace=True)
        #df_sentiments.rename(columns={0: "headline_m2_sentiment_0"}, inplace=True)
        return df_sentiments
    
    @classmethod
    def get_cluster(cls, dfs, inputs):
        # ニュースとそのBERT特徴量を取り出す & 結合
        articles = SentimentGenerator.target_article
        weekly_group = SentimentGenerator._build_weekly_group(articles)
        articles = articles.groupby(weekly_group).apply(lambda x: x[:])
        features = SentimentGenerator.features['headline']
        features.columns = [f'feature_{i}' for i in features.columns]

        # ニュースと業種を紐づける
        articles = pd.concat([articles, features], axis=1).copy()
        articles.dropna(subset=['Local Code'], inplace=True)

        # K-means
        predict = cls.models['kmeans'].predict(articles.loc[:, articles.columns.str.contains('feature_')])
        articles['cluster'] = predict

        return articles[['Local Code', 'cluster']]

    @classmethod
    def strategy(cls, df_sentiments, df_cluster):
        df_target = df_cluster.copy()
        index_lev1 = list(set(df_target.index.get_level_values(0)))
        print(index_lev1)

        # 基準化
        df_sentiments = (df_sentiments - cls.df_sentiment_dist.mean()) / cls.df_sentiment_dist.std()
      
        for i, ind in enumerate(index_lev1):
            # 週次の投資対象一覧を取得
            df_weekly_target = df_target.loc[ind].copy()

            ###################
            # クラスター番号によるスクリーニング
            ###################
            # クラスター7が5銘柄より少ない場合は、クラスター4だけ除外する
            if  len(df_weekly_target[df_weekly_target['cluster'] == 7]['Local Code'].unique()) < 5:
                df_weekly_target = df_weekly_target[df_weekly_target['cluster'] != 4].copy()
            else:
                df_weekly_target = df_weekly_target[df_weekly_target['cluster'] == 7].copy()
            # 銘柄数が足りない場合は、クラスター番号によるスクリーニングをしない
            if len(df_weekly_target) < 5:
                df_weekly_target = df_target.loc[ind]

            ###################
            # 業種区分によるスクリーニング
            ###################
            df_target_tmp = df_weekly_target.copy()
            del_sector = [2, 11, 12, 15]
            df_weekly_target = df_weekly_target[~df_weekly_target['sector'].isin(del_sector)].copy()
            if len(df_weekly_target['Local Code'].unique()) < 5:
                df_weekly_target = df_target.copy()

            ###################
            # 各業種への投資比率
            ###################
            # セクターへの投資比率をセンチメントスコアから決定する
            weekly_sentiment = df_sentiments.iloc[i, df_weekly_target['sector'].unique().tolist()] + 1 # +1することでマイナスを解消(ウェイト計算のため)
            weekly_sentiment[weekly_sentiment < 0]  = 0 # 稀にマイナスの場合があるため、0にしておく
            sector_weights = weekly_sentiment / weekly_sentiment.sum()
            # 各セクターの銘柄数を算出し、各セクターの銘柄投資比率(均等)を決定する
            stock_num = df_weekly_target.groupby('sector').count()['Local Code']
            df_weekly_target['weight'] = df_weekly_target['sector'].apply(lambda x: sector_weights[x] / stock_num[x])

            ###################
            # 現金比率
            ###################
            invest_total = 1000000 # 100万円

            # 投資対象ユニバースの予測スコアから現金比率を決定する
            weekly_market_sentiment = df_sentiments.iloc[i, 0]
            z = (cls.df_sentiment_dist[0] - cls.df_sentiment_dist.mean()[0]) / ss.df_sentiment_dist.std()[0] 
            p = np.percentile(z, [25, 50, 75])
            tile = np.digitize(weekly_market_sentiment, p) # 0～3の値:値が高いほど、投資比率を高くする
            # (メモ)
            # 投資比率は適当(0.5基準の10%刻み)。現金資産もあったほうが良いとも割れるため、少なくとも20%は充てるようにした。
            if tile == 0:
                invest_total = 0.5 * invest_total # 50%現金
            elif tile == 1:
                invest_total = 0.6 * invest_total # 60%現金
            elif tile == 2:
                invest_total = 0.7 * invest_total # 70%現金
            else:
                invest_total = 0.8 * invest_total # 80％投資

            df_weekly_target['budget'] = df_weekly_target['weight'] * invest_total

            # 直近のニュースを優先とする(ウェイトの調整は諦め、ソートで対応)
            df_weekly_target.sort_index(ascending=False, inplace=True)

            # 日付を(Y, W)を戻す
            df_weekly_target.reset_index(inplace=True)
            df_weekly_target.index = pd.MultiIndex.from_tuples([ind] * len(df_weekly_target)) 

            # 保存
            if i == 0:
                df = df_weekly_target.copy()
            else:
                df = pd.concat([df, df_weekly_target], axis=0)
      
        return df

    @classmethod
    def predict(
        cls,
        inputs,
        start_dt=TEST_START,
        load_data=["stock_list", 
                   #"tdnet", 
                   #"purchase_date",
                   ],
    ):
        """Predict method
        Args:
            inputs (dict[str]): paths to the dataset files
            codes (list[int]): traget codes
            start_dt (str): specify target purchase date
            load_data (list[str]): list of data to load
        Returns:
            str: Inference for the given input.
        """
        # データ読み込み
        if cls.dfs is None:
            print("[+] load data")
            cls.get_dataset(inputs, load_data)
            cls.get_codes(cls.dfs)

        # purchase_date が存在する場合は予測対象日を上書き
        if "purchase_date" in cls.dfs.keys():
            # purchase_dateの最も古い日付を設定
            start_dt = cls.dfs["purchase_date"].sort_values("Purchase Date").iloc[0, 0]

        # 日付型に変換
        start_dt = pd.Timestamp(start_dt)
        # 予測対象日の月曜日日付が指定されているため
        # 特徴量の抽出に使用する1週間前の日付に変換します
        start_dt -= pd.Timedelta("7D")
        # 文字列型に戻す
        start_dt = start_dt.strftime("%Y-%m-%d")

        ###################
        # センチメント情報取得
        ###################
        # ニュース見出しデータへのパスを指定
        df_sentiments = cls.get_sentiment(inputs, start_dt=start_dt)
        #
        # 金曜日日付に変更
        df_sentiments.index = df_sentiments.index + pd.Timedelta("4D")
        
        ###################
        # K-means(クラスタリング)
        ###################
        # 各ニュースにクラスター番号を付ける
        df_cluster = cls.get_cluster(cls.dfs, inputs)
        #
        # 業種区分を紐づける(ニュースが複数銘柄のものは欠落する)←欠落しないように対応すべきだったのかも
        df_sector = ss.dfs['stock_list'][['Local Code', '17 Sector(Code)']]
        df_cluster['sector'] = df_cluster['Local Code'].apply(lambda x: df_sector[df_sector['Local Code'].astype(str) == str(x)]['17 Sector(Code)'].values)
        df_cluster['sector'] = df_cluster['sector'].apply(lambda x: x[0] if len(x) != 0 else np.nan) # 業種区分が無い場合、[]となるためnanに変換(力技過ぎる...)
        df_cluster.dropna(subset=['sector'], inplace=True)

        ###################
        # 銘柄選定
        ###################
        df = cls.strategy(df_sentiments, df_cluster)
        
        # 結果を以下のcsv形式で出力する
        # 1列目:date
        # 2列目:Local Code
        # 3列目:budget
        # headerあり、2列目3列目はint64

        # 月曜日日付(ニュース公表の週初)に変更後、1週間ずらす
        df.index = df.index.map(lambda x: cls.transform_yearweek_to_monday(x[0], x[1]))
        df.index = df.index + pd.offsets.Week(1)

        # 出力用に調整
        df.index.name = "date"
        df.reset_index(inplace=True)
        df['Local Code'] = df['Local Code'].astype(int)
        df['budget'] = df['budget'].astype(int)

        # 出力対象列を定義
        output_columns = ["date", "Local Code", "budget"]

        out = io.StringIO()
        df.to_csv(out, header=True, index=False, columns=output_columns)

        # csvで保存しておく
        df[output_columns].to_csv('./result.csv')

        return out.getvalue()

In [3]:
dataset_dir = '../data/'
# 入力パラメーターを設定します。ランタイム環境での実行時と同一フォーマットにします
inputs = {
    "stock_list": f"{dataset_dir}/stock_list.csv.gz",
    "stock_price": f"{dataset_dir}/stock_price.csv.gz",
    "stock_fin": f"{dataset_dir}/stock_fin.csv.gz",
    "stock_fin_price": f"{dataset_dir}/stock_fin_price.csv.gz",
    # ニュースデータ
    "tdnet": f"{dataset_dir}/tdnet.csv.gz",
    "disclosureItems": f"{dataset_dir}/disclosureItems.csv.gz",
    "nikkei_article": f"{dataset_dir}/nikkei_article.csv.gz",
    "article": f"{dataset_dir}/article.csv.gz",
    "industry": f"{dataset_dir}/industry.csv.gz",
    "industry2": f"{dataset_dir}/industry2.csv.gz",
    #"region": f"{dataset_dir}/region.csv.gz",
    #"theme": f"{dataset_dir}/theme.csv.gz",
    # 目的変数データ
    #"stock_labels": f"{dataset_dir}/stock_labels.csv.gz",
}

In [6]:
ss = ScoringService()
ss.get_dataset(inputs, ["stock_list"])
ss.get_model()

[+] Set Device: CPU
[+] Built feature extractor
[+] Built bert tokenizer
[+] Set Device: CPU
[+] Model is loaded
[+] load prepared sentiment: ../model/headline_features/LSTM_sentiment.pkl


True

In [14]:
# 詳細な確認用
ss.get_dataset(inputs, ["stock_list"])
ss.get_codes(ss.dfs)


start_dt = '2020/12/30'
# 日付型に変換
start_dt = pd.Timestamp(start_dt)
# 予測対象日の月曜日日付が指定されているため
# 特徴量の抽出に使用する1週間前の日付に変換します
start_dt -= pd.Timedelta("7D")
# 文字列型に戻す
start_dt = start_dt.strftime("%Y-%m-%d")

df_sentiments = ss.get_sentiment(inputs, start_dt=start_dt)
# 金曜日日付に変更
df_sentiments.index = df_sentiments.index + pd.Timedelta("4D")

# 各ニュースにクラスター番号を付ける
df_cluster = ss.get_cluster(ss.dfs, inputs)

# 業種区分を紐づける(ニュースが複数銘柄のものは欠落する)←欠落しないように対応すべきだったのかも
df_sector = ss.dfs['stock_list'][['Local Code', '17 Sector(Code)']]
df_cluster['sector'] = df_cluster['Local Code'].apply(lambda x: df_sector[df_sector['Local Code'].astype(str) == str(x)]['17 Sector(Code)'].values)
df_cluster['sector'] = df_cluster['sector'].apply(lambda x: x[0] if len(x) != 0 else np.nan) # 業種区分が無い場合、[]となるためnanに変換(力技過ぎる...)
df_cluster.dropna(subset=['sector'], inplace=True)

###################
# 銘柄選定
###################
df = ss.strategy(df_sentiments, df_cluster)



# まとめて確認用
#ss.predict(inputs, '2020-12-30') # 2021-01-07 2020-12-30

../src/SentimentGenerator.py:303: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  return pd.Series(list(zip(df.index.year, df.index.week)), index=df.index)



[(2020, 52), (2020, 53)]


In [34]:
df = ss.strategy(df_sentiments, df_cluster)
df

[(2020, 52), (2020, 53)]


publish_datetime Local Code  cluster  sector    weight  \
2020 52 2020-12-25 22:54:47+09:00       9432        7    10.0  0.011128   
     52 2020-12-25 21:00:00+09:00       7974        7    10.0  0.011128   
     52 2020-12-25 20:49:09+09:00       6502        7     9.0  0.087187   
     52 2020-12-25 20:04:47+09:00       4004        7     4.0  0.096537   
     52 2020-12-25 19:29:15+09:00       4680        7    10.0  0.011128   
     52 2020-12-25 19:25:00+09:00       7261        7     6.0  0.076789   
     52 2020-12-25 19:00:00+09:00       3880        7     4.0  0.096537   
     52 2020-12-25 16:09:25+09:00       8848        7    17.0  0.085252   
     52 2020-12-25 11:34:20+09:00       6502        7     9.0  0.087187   
     52 2020-12-25 10:25:09+09:00       3137        7    14.0  0.000952   
     52 2020-12-25 05:26:41+09:00       9983        7    14.0  0.000952   
     52 2020-12-25 03:00:00+09:00       6448        7     9.0  0.087187   
     52 2020-12-24 21:30:00+09:00       8473        7    16.0  0.048564   
     52 2020-12-24 20:50:46+09:00       7205        7     6.0  0.076789   
     52 2020-12-24 16:44:15+09:00       2484        7    10.0  0.011128   
     52 2020-12-23 21:24:11+09:00       7532        7    14.0  0.000952   
     52 2020-12-23 17:52:56+09:00       7203        7     6.0  0.076789   
     52 2020-12-23 16:00:00+09:00       7342        7    16.0  0.048564   
     52 2020-12-23 16:00:00+09:00       8848        7    17.0  0.085252   
     53 2020-12-30 17:09:01+09:00       9424        7    10.0  0.036135   
     53 2020-12-29 23:30:00+09:00       9984        7    10.0  0.036135   
     53 2020-12-28 19:16:01+09:00       7860        7    10.0  0.036135   
     53 2020-12-28 18:56:21+09:00       6849        7     9.0  0.315285   
     53 2020-12-28 18:03:25+09:00       8227        7    14.0  0.112306   
     53 2020-12-28 17:05:09+09:00       9602        7    10.0  0.036135   
     53 2020-12-28 14:22:55+09:00       7012        7     6.0  0.258462   
     53 2020-12-28 02:00:00+09:00       2503        7     1.0  0.169408   

                budget  
2020 52    5563.838873  
     52    5563.838873  
     52   43593.476216  
     52   48268.634826  
     52    5563.838873  
     52   38394.321998  
     52   48268.634826  
     52   42625.803500  
     52   43593.476216  
     52     476.060008  
     52     476.060008  
     52   43593.476216  
     52   24282.081053  
     52   38394.321998  
     52    5563.838873  
     52     476.060008  
     52   38394.321998  
     52   24282.081053  
     52   42625.803500  
     53   21680.818498  
     53   21680.818498  
     53   21680.818498  
     53  189170.926809  
     53   67383.742332  
     53   21680.818498  
     53  155077.403784  
     53  101644.653082

In [35]:
df.index.map(lambda x: ss.transform_yearweek_to_monday(x[0], x[1]))

DatetimeIndex(['2020-12-21', '2020-12-21', '2020-12-21', '2020-12-21',
               '2020-12-21', '2020-12-21', '2020-12-21', '2020-12-21',
               '2020-12-21', '2020-12-21', '2020-12-21', '2020-12-21',
               '2020-12-21', '2020-12-21', '2020-12-21', '2020-12-21',
               '2020-12-21', '2020-12-21', '2020-12-21', '2020-12-28',
               '2020-12-28', '2020-12-28', '2020-12-28', '2020-12-28',
               '2020-12-28', '2020-12-28', '2020-12-28'],
              dtype='datetime64[ns]', freq=None)

In [36]:
df.index = df.index.map(lambda x: ss.transform_yearweek_to_monday(x[0], x[1]))
df.index = df.index + pd.offsets.Week(1)
df

,publish_datetime,Local Code,cluster,sector,weight,budget
2020-12-28,2020-12-25 22:54:47+09:00,9432,7,10.0,0.011128,5563.838873
2020-12-28,2020-12-25 21:00:00+09:00,7974,7,10.0,0.011128,5563.838873
2020-12-28,2020-12-25 20:49:09+09:00,6502,7,9.0,0.087187,43593.476216
2020-12-28,2020-12-25 20:04:47+09:00,4004,7,4.0,0.096537,48268.634826
2020-12-28,2020-12-25 19:29:15+09:00,4680,7,10.0,0.011128,5563.838873
2020-12-28,2020-12-25 19:25:00+09:00,7261,7,6.0,0.076789,38394.321998
2020-12-28,2020-12-25 19:00:00+09:00,3880,7,4.0,0.096537,48268.634826
2020-12-28,2020-12-25 16:09:25+09:00,8848,7,17.0,0.085252,42625.803500
2020-12-28,2020-12-25 11:34:20+09:00,6502,7,9.0,0.087187,43593.476216
2020-12-28,2020-12-25 10:25:09+09:00,3137,7,14.0,0.000952,476.060008


In [37]:
# 出力用に調整
df.index.name = "date"
df.reset_index(inplace=True)
df['Local Code'] = df['Local Code'].astype(int)
df['budget'] = df['budget'].astype(int)

In [38]:
df

,date,publish_datetime,Local Code,cluster,sector,weight,budget
0,2020-12-28,2020-12-25 22:54:47+09:00,9432,7,10.0,0.011128,5563
1,2020-12-28,2020-12-25 21:00:00+09:00,7974,7,10.0,0.011128,5563
2,2020-12-28,2020-12-25 20:49:09+09:00,6502,7,9.0,0.087187,43593
3,2020-12-28,2020-12-25 20:04:47+09:00,4004,7,4.0,0.096537,48268
4,2020-12-28,2020-12-25 19:29:15+09:00,4680,7,10.0,0.011128,5563
5,2020-12-28,2020-12-25 19:25:00+09:00,7261,7,6.0,0.076789,38394
6,2020-12-28,2020-12-25 19:00:00+09:00,3880,7,4.0,0.096537,48268
7,2020-12-28,2020-12-25 16:09:25+09:00,8848,7,17.0,0.085252,42625
8,2020-12-28,2020-12-25 11:34:20+09:00,6502,7,9.0,0.087187,43593
9,2020-12-28,2020-12-25 10:25:09+09:00,3137,7,14.0,0.000952,476


In [39]:
# 出力対象列を定義
output_columns = ["date", "Local Code", "budget"]

# csvで保存しておく
df[output_columns].to_csv('./result.csv')

In [41]:
df_sentiments

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
index,,,,,,,,,,,,,,,,,,
2020-12-25,0.550833,0.524002,0.515939,0.533248,0.595576,0.485385,0.360859,0.420584,0.502232,0.490664,0.493269,0.606377,0.455933,0.476527,0.482201,0.391926,0.540757,0.442580
2021-01-01,0.551386,0.524057,0.516359,0.533394,0.595752,0.485686,0.360934,0.420645,0.502448,0.490751,0.493586,0.606526,0.455858,0.476985,0.482737,0.391746,0.541053,0.442803


,cluster,sector
Local Code,,
2484,1,1
2503,1,1
3137,1,1
3880,1,1
4004,1,1
4680,1,1
6448,1,1
6502,2,2
6849,1,1
